In [34]:
from datetime import datetime
startTime = datetime.now()

filepath = 'input.txt'

INSTRUCTION_JUMPS = {'01': 4, '02': 4, '03': 2, '04': 2, '05': 3, '06': 3, '07': 4, '08': 4, '09': 2}
MODES = {'0': lambda i, n, data, relative_base: data[data[i+n]], 
         '1': lambda i, n, data, relative_base: data[i+n],
         '2': lambda i, n, data, relative_base: data[data[i+n] + relative_base]}

class intcode():
    def __init__(self, data):
        self.data = data + [0] * 1000
        self.i = 0
        self.num_inputs = 0
        self.relative_base = 0
        self.inputs = []
        self.internal_program = self.run_program()
        
    def add_input(self, addition):
        self.inputs.append(addition)
    
    def next_out(self):
        
        self.output = next(self.internal_program)
        return self.output
        
    def run_program(self):
        while self.data[self.i] != 99:
            

            self.opcode = str(self.data[self.i])[-2:].zfill(2)
            self.mode = str(self.data[self.i])[:-2].zfill(3)

            if self.opcode in ('01', '02', '04', '05', '06', '07', '08', '09'):
                self.p1 = MODES[self.mode[2]](self.i, 1, self.data, self.relative_base)     
            if self.opcode in ('01', '02', '05', '06', '07', '08'):
                self.p2 = MODES[self.mode[1]](self.i, 2, self.data, self.relative_base)

            if self.opcode in ['03']:
                if self.mode[2] == '2':
                    self.p3 = self.data[self.i+1] + self.relative_base
                else:
                    self.p3 = self.data[self.i+1]
                self.data = self.data + [0] * (self.p3+1-len(self.data))                  

            if self.opcode in ['01', '02', '07', '08']:
                if self.mode[0] == '2':
                    self.p3 = self.data[self.i+3] + self.relative_base
                else:
                    self.p3 = self.data[self.i+3]
                self.data = self.data + [0] * (self.p3+1-len(self.data))

            if self.opcode == '01':
                self.data[self.p3] = self.p1 + self.p2
            elif self.opcode == '02':
                self.data[self.p3] = self.p1 * self.p2
            elif self.opcode == '03':
                self.data[self.p3] = self.inputs[self.num_inputs]
                self.num_inputs += 1
            elif self.opcode == '04':
                #self.outputs.append(self.p1)
                yield self.p1 
            elif self.opcode == '05' and self.p1 != 0:
                self.i = self.p2
                continue
            elif self.opcode == '06' and self.p1 == 0:
                self.i = self.p2
                continue 
            elif self.opcode == '07':
                self.data[self.p3] = int(self.p1 < self.p2) 
            elif self.opcode == '08':
                self.data[self.p3] = int(self.p1 == self.p2)
            elif self.opcode == '09':
                self.relative_base += self.p1
            else:
                pass

            self.i += INSTRUCTION_JUMPS[self.opcode]
            
        #yield 'HALTED'

In [1]:
# 19354928
# 1141997803

In [35]:
with open(filepath, 'r') as f:
    for line in f.readlines():
        data = [int(i) for i in line.strip().split(',')]

instructions = [[], []]
#for n, file in enumerate(['p1_springscript.txt', 'p2_springscript.txt']):
for n, file in enumerate(['p1_springscript.txt']):
    with open(file, 'r') as f:
        for line in f.readlines():
            if line == '\n':
                continue
            instructions[n] = instructions[n] + [ord(char) for char in line.strip()] + [10]

# TROUBLESHOOTING PRINTOUT
'''try:
    while 1:
        line = ''
        while 1:
            next_char = next(program)
            if next_char == None:
                break
            elif next_char != 10:
                line = line + chr(next_char)
            else:
                print(line)
                line = ''
except ValueError:
    print(next_char)'''

program = intcode(data[:])

for i in instructions[0]:
    program.add_input(i)
    
try:
    while 1:
        last_char = program.next_out()
except StopIteration:
    print(last_char)
        
print(str(datetime.now() - startTime)[:-3])

19354928
0:00:01.489
